In [ ]:

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.utils import class_weight


try:
    from kerastuner.tuners import RandomSearch
    USE_TUNER = True
except ImportError:
    USE_TUNER = False
    print("Keras Tuner not installed, will train with default parameters")


BASE_DIR = "/Users/ishanlahiru/Documents/grocery_classifier/dataset"  
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "validation")
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 15
NUM_CLASSES = 5
LEARNING_RATE = 1e-4



train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.15,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3],
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)


y_train = train_generator.classes  
weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(weights))
print("Class weights:", class_weights)


def build_model(hp=None):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
    base_model.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)

   
    if hp:
        x = Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1))(x)
        x = Dense(hp.Int('dense_units', 64, 256, step=64), activation='relu')(x)
        lr = hp.Choice('learning_rate', [1e-3, 1e-4, 1e-5])
    else:
        x = Dropout(0.3)(x)
        x = Dense(128, activation='relu')(x)
        lr = LEARNING_RATE

    output = Dense(NUM_CLASSES, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=output)
    model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model


if USE_TUNER:
    print("Using Keras Tuner for hyperparameter search...")
    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=10,
        executions_per_trial=1,
        directory='tuner_dir',
        project_name='grocery_cnn'
    )

    tuner.search(train_generator, validation_data=val_generator, epochs=10, class_weight=class_weights)
    best_model = tuner.get_best_models(num_models=1)[0]
else:
    print("Training with default parameters...")
    best_model = build_model()
    best_model.fit(
        train_generator,
        epochs=EPOCHS,
        validation_data=val_generator,
        class_weight=class_weights
    )


MODEL_PATH = os.path.join(BASE_DIR, "best_grocery_model.h5")
best_model.save(MODEL_PATH)
print(f" Model saved to {MODEL_PATH}")


Keras Tuner not installed, will train with default parameters
Found 397 images belonging to 5 classes.
Found 84 images belonging to 5 classes.
Class weights: {0: np.float64(0.9452380952380952), 1: np.float64(0.9452380952380952), 2: np.float64(0.7561904761904762), 3: np.float64(1.2806451612903227), 4: np.float64(1.2806451612903227)}
Training with default parameters...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


/Users/ishanlahiru/miniforge3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 195ms/step - accuracy: 0.3526 - loss: 1.5698 - val_accuracy: 0.6071 - val_loss: 1.1972
Epoch 2/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 168ms/step - accuracy: 0.6574 - loss: 1.0995 - val_accuracy: 0.8929 - val_loss: 0.7908
Epoch 3/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 162ms/step - accuracy: 0.8086 - loss: 0.7708 - val_accuracy: 0.9643 - val_loss: 0.5188
Epoch 4/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.8741 - loss: 0.5627 - val_accuracy: 1.0000 - val_loss: 0.3346
Epoch 5/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 164ms/step - accuracy: 0.9295 - loss: 0.4191 - val_accuracy: 1.0000 - val_loss: 0.2395
Epoch 6/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 172ms/step - accuracy: 0.9496 - loss: 0.3132 - val_accuracy: 1.0000 - val_loss: 0.1603
Epoch 7/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 165ms/step - accuracy: 0.9471 - loss: 0.2742 - val_accuracy: 1.0000 - val_loss: 0.1242
Epoch 8/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 179ms/step - accuracy: 0.9622 - loss: 0.2233 - val_accuracy: 1.

✅ Model saved to /Users/ishanlahiru/Documents/grocery_classifier/dataset/best_grocery_model.h5
